In [130]:
import os
import yfinance as yf
import pandas as pd
import streamlit as st
import plotly.graph_objects as go
from datetime import datetime, timedelta
from sbi_access import get_sbi_holdings
from algo_trade import algo_trade
import csv
import requests
from io import StringIO
from dotenv import load_dotenv

import copy
import yfinance as yf
import pandas as pd
import numpy as np
import streamlit as st
import os
import requests
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from datetime import datetime, timedelta
from typing import List
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from autogluon.tabular import TabularPredictor

In [131]:
if not os.path.exists('./data'):
    os.makedirs('./data')

def download_stock_data(symbols: List[str], target_symbols: List[str], start_date: str, end_date: str):
    all_symbols = list(symbols + target_symbols)
    data = pd.DataFrame()
    remove_symbols = []
    for symbol in list(set(all_symbols)):
        file_name = f"./data/{symbol}.csv"
        if os.path.isfile(file_name):
            stock_data = pd.read_csv(file_name, index_col=0, parse_dates=True)
            stock_data.index = pd.to_datetime(stock_data.index)
            last_date = stock_data.index[-1]
            if last_date >= pd.Timestamp(datetime.now()):
                new_data = yf.download(symbol, start=last_date.date() + timedelta(days=1), end=end_date, interval='1d')
                if not new_data.empty and pd.isna(new_data['Adj Close'].iloc[-1]):
                    stock_data = stock_data.append(new_data)
                    stock_data = stock_data[~stock_data.index.duplicated(keep='last')]
                    with open(file_name, mode='w') as f:
                        stock_data.to_csv(f)
                else:
                    remove_symbols.append(symbol)
            else:
                pass
        else:
            stock_data = yf.download(symbol, start=start_date, end=end_date, interval='1d')
            if not stock_data.empty and pd.isna(stock_data['Adj Close'].iloc[-1]):
                with open(file_name, mode='w') as f:
                    stock_data.to_csv(f)
            else:
                remove_symbols.append(symbol)
        try:
            data[symbol] = stock_data['Adj Close']
        except Exception as e:
            remove_symbols.append(symbol)
            # print(symbol, e)
            pass
    for item in list(set(remove_symbols)):
        if item in symbols:
            symbols.remove(item)
        if item in target_symbols:
            target_symbols.remove(item)
    with open("./feat_symbols.txt", "w") as f:
        for symbol in symbols:
            f.write(f"{symbol}\n")
    with open("./target_symbols.txt", "w") as f:
        for symbol in target_symbols:
            f.write(f"{symbol}\n")

    return data, symbols, target_symbols

def prepare_data(data, symbols, target_symbols, shift=1):
    prepared_data = {}
    for target_symbol in target_symbols:
        stock_data = pd.DataFrame(data[target_symbol])
        stock_data[f'{target_symbol}_lag_{shift}'] = data[target_symbol].shift(shift, fill_value=0)
        for symbol in symbols:
            if symbol != target_symbol:
                stock_data[f'{symbol}_lag_{shift}'] = data[symbol].shift(shift, fill_value=0)
        prepared_data[target_symbol] = stock_data
    
    for key in prepared_data.keys():
        # shiftの先頭行を排除
        prepared_data[key] = prepared_data[key].iloc[shift:,:]

        # NoneをNaNに置換
        prepared_data[key] = prepared_data[key].replace(to_replace=['None', 'null', 'nan', 'NA'], value=np.nan)

        # 欠損値NaNを過去最新の値で埋める
        prepared_data[key] = prepared_data[key].fillna(method="ffill")
        prepared_data[key] = prepared_data[key].dropna(axis=1)            

        if prepared_data[key].isnull().any().any():
            st.write(f"{key} : NaN values found.")
    
    return prepared_data

def feature_selection(X, y):
    rf = RandomForestRegressor(n_jobs=-1, max_depth=5)
    feature_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=42)
    feature_selector.fit(X.values, y.values.ravel())
    selected_features = X.columns[feature_selector.support_].tolist()
    return selected_features

def load_selected_features(target_symbol, lag):
    file_name = f"./feature_selection/{target_symbol}_lag{lag}_selected_features.txt"
    if os.path.isfile(file_name):
        with open(file_name, "r") as f:
            selected_features = [line.strip() for line in f.readlines()]
        return selected_features
    return None

def save_selected_features(target_symbol, selected_features, lag):
    file_name = f"./feature_selection/{target_symbol}_lag{lag}_selected_features.txt"
    with open(file_name, "w") as f:
        for feature in selected_features:
            f.write(f"{feature}\n")
            
def train_and_test(data, symbols, target_symbols, original_data, future_days=1):
    predictions = {}
    actuals = {}
    future_predictions = {}

    for target_symbol in target_symbols:
        X = data[target_symbol].drop([target_symbol], axis=1)
        y = data[target_symbol][target_symbol]

        # Perform feature selection using Boruta
        selected_features = load_selected_features(target_symbol, future_days)
        if not selected_features:
            selected_features = feature_selection(X, y)
            save_selected_features(target_symbol, selected_features, future_days)
        
        X = X[selected_features]
        X[target_symbol] = y

        # Split data for final evaluation
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        _y_train, y_test = y[:train_size], y[train_size:]
        
        # Initialize the Autogluon TabularPredictor
        predictor = TabularPredictor(label=target_symbol, path=f'C:/Users/rodin/work/stock_trade/', problem_type='regression')

        # Fit the model with cross-validation and excluding CatBoost
        predictor.fit(train_data=X_train, presets='best_quality', time_limit=60)

        predictions[target_symbol] = pd.DataFrame(predictor.predict(X_test).values, index=y_test.index, columns=["prediction"])

        actuals[target_symbol] = pd.DataFrame(y_test.values, index=y_test.index, columns=["actual"])

        future_dates = [data[target_symbol].index[-1] + timedelta(days=i) for i in range(1, future_days + 1)]

        # Initialize future_df with columns for all symbols
        future_df = pd.DataFrame(index=future_dates, columns=X.columns).drop([target_symbol], axis=1)

        for column in future_df.columns:
            symbol, _, lag = column.split('_')
            lag = int(lag)
            future_df.loc[:, column] = original_data[symbol].iloc[-lag:].values
    
        # Predict future prices for the target symbol and save the results
        future_predictions[target_symbol] = pd.DataFrame(predictor.predict(future_df[selected_features]).values, index=future_dates[:len(future_df)], columns=["prediction"])

    return predictions, actuals, future_predictions

def algo_trade(symbols, target_symbols, years, shift):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years*365)
    data, symbols, target_symbols = download_stock_data(symbols, target_symbols, start_date, end_date)
    original_data = copy.deepcopy(data)
    prepared_data = prepare_data(data, symbols, target_symbols, shift=shift)
    
    predictions, actuals, future_predictions = train_and_test(prepared_data, symbols, target_symbols, original_data, future_days=shift)

    return predictions, actuals, future_predictions


In [132]:
# 銘柄リストを取得する関数
def get_symbol_list(file):
    with open(file, "r") as f:
        symbols = [line.strip() for line in f.readlines()]
    return symbols
    
# スクレイピングを行う関数
def get_stock_info():
    stock_table, balance = get_sbi_holdings(username, password)
    asset = round(stock_table["評価額"].sum()/10000)
    balance = round(balance/10000)
    return stock_table, asset, balance

# スクレイピングしたデータをキャッシュする
def cached_data():
    return get_stock_info()

def fetch_data(symbols, interval, start_date, end_date):
    data = {}
    for symbol in symbols:
        stock_data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        data[symbol] = stock_data
    return data

def display_charts(data, symbols, interval):
    for symbol in symbols:
        st.subheader(f"{symbol} {interval} Chart")

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data[symbol].index, y=data[symbol]['Close'],
                    mode='lines',
                    name='Close Price'))
        fig.update_layout(
            title=f"{symbol} {interval} Chart",
            xaxis_title="Date",
            yaxis_title="Close Price"
        )

        st.plotly_chart(fig)

def calculate_mape(actuals, predictions):
    mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100
    return mape

def display_prediction(target_symbols, predictions, actuals, future_predictions):
    # クロスバリデーションのもっともよかったモデルの予測と実績の対比と、
    # 一週間分の将来予測を時系列グラフにして表示
    for target_symbol in target_symbols:
        # 時系列プロットとy-yプロットのカラムを作成

        # 時系列プロット
        fig = go.Figure()

        # 時系列プロットの幅を900に設定
        fig.update_layout(width=850)

        fig.add_trace(go.Scatter(x=actuals[target_symbol].index,
                                y=actuals[target_symbol]['actual'],
                                mode='lines',
                                name='Actual',
                                line=dict(color='black')))
        fig.add_trace(go.Scatter(x=predictions[target_symbol].index,
                                y=predictions[target_symbol]['prediction'],
                                mode='lines',
                                name='Predicted',
                                line=dict(color='red', dash='3px,2px', width=1.4)))

        # Add future predictions to the plot
        future_dates = future_predictions[target_symbol].index
        fig.add_trace(go.Scatter(x=future_dates,
                                y=future_predictions[target_symbol]['prediction'],
                                mode='lines',
                                name='Forecasted',
                                line=dict(color='green', dash='3px,2px', width=1.4)))

        fig.update_layout(title=f"{target_symbol} Predictions vs Actuals vs Forecasted",
                        xaxis_title="Date",
                        yaxis_title="Close Price")

        # y-yプロット
        fig_yy = go.Figure()

        # y-yプロットの幅を300に設定
        fig_yy.update_layout(width=350)

        fig_yy.add_trace(go.Scatter(x=actuals[target_symbol]['actual'],
                                    y=predictions[target_symbol]['prediction'],
                                    mode='markers',
                                    name='Predicted vs Actual',
                                    marker=dict(color='blue', size=5)))

        # y=yの点線を追加
        fig_yy.add_shape(
            type='line',
            x0=min(actuals[target_symbol]['actual']),
            x1=max(actuals[target_symbol]['actual']),
            y0=min(actuals[target_symbol]['actual']),
            y1=max(actuals[target_symbol]['actual']),
            yref='y',
            xref='x',
            line=dict(color='black', dash='dot')
        )

        # MAPEの計算
        mape = calculate_mape(actuals[target_symbol]['actual'], predictions[target_symbol]['prediction'])

        fig_yy.update_layout(title=f"{target_symbol} Y-Y Plot (MAPE: {mape:.2f}%)",
                            xaxis_title="Actual",
                            yaxis_title="Predicted")
        fig.show()

In [133]:
years = 5 # 学習データの期間
shift = 5 # n 期先予測

symbols = get_symbol_list("feat_symbols.txt")
target_symbols = get_symbol_list("target_symbols.txt")

predictions, actuals, future_predictions = algo_trade(symbols, target_symbols, years, shift)

C:\Users\rodin\AppData\Local\Temp\ipykernel_22952\1505525427.py:33: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\rodin\AppData\Local\Temp\ipykernel_22952\1505525427.py:33: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\rodin\AppData\Local\Temp\ipykernel_22952\1505525427.py:33: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

In [134]:
display(predictions)
display(actuals)
display(future_predictions)

{'4584.T':             prediction
 Date                  
 2022-04-22  466.021271
 2022-04-25  462.821228
 2022-04-26  457.203979
 2022-04-27  470.194519
 2022-04-28  471.487915
 ...                ...
 2023-04-17  589.687744
 2023-04-18  596.604553
 2023-04-19  601.088257
 2023-04-20  601.071960
 2023-04-21  600.390015
 
 [251 rows x 1 columns],
 '1557.T':               prediction
 Date                    
 2022-04-22  56271.507812
 2022-04-25  56090.335938
 2022-04-26  55976.402344
 2022-04-27  55956.234375
 2022-04-28  55515.574219
 ...                  ...
 2023-04-17  47400.000000
 2023-04-18  48552.910156
 2023-04-19  48492.324219
 2023-04-20  48840.617188
 2023-04-21  48394.453125
 
 [251 rows x 1 columns],
 '8789.T':             prediction
 Date                  
 2022-04-22   43.185486
 2022-04-25   42.719479
 2022-04-26   43.433487
 2022-04-27   43.614517
 2022-04-28   43.143738
 ...                ...
 2023-04-17   62.194145
 2023-04-18   63.224930
 2023-04-19   63.114830
 2

{'4584.T':             actual
 Date              
 2022-04-22   469.0
 2022-04-25   464.0
 2022-04-26   458.0
 2022-04-27   438.0
 2022-04-28   467.0
 ...            ...
 2023-04-17   249.0
 2023-04-18   254.0
 2023-04-19   253.0
 2023-04-20   251.0
 2023-04-21   244.0
 
 [251 rows x 1 columns],
 '1557.T':              actual
 Date               
 2022-04-22  55950.0
 2022-04-25  54320.0
 2022-04-26  54940.0
 2022-04-27  53450.0
 2022-04-28  54730.0
 ...             ...
 2023-04-17  55370.0
 2023-04-18  55640.0
 2023-04-19  55630.0
 2023-04-20  55640.0
 2023-04-21  55080.0
 
 [251 rows x 1 columns],
 '8789.T':             actual
 Date              
 2022-04-22    42.0
 2022-04-25    41.0
 2022-04-26    43.0
 2022-04-27    41.0
 2022-04-28    42.0
 ...            ...
 2023-04-17    65.0
 2023-04-18    67.0
 2023-04-19    67.0
 2023-04-20    67.0
 2023-04-21    69.0
 
 [251 rows x 1 columns],
 '1893.T':                 actual
 Date                  
 2022-04-22  614.297241
 2022-04-25  5

{'4584.T':             prediction
 2023-04-22  599.554871
 2023-04-23  602.644409
 2023-04-24  603.763489
 2023-04-25  607.525696
 2023-04-26  607.986572,
 '1557.T':               prediction
 2023-04-22  49160.675781
 2023-04-23  48538.625000
 2023-04-24  48417.152344
 2023-04-25  48564.863281
 2023-04-26  48074.449219,
 '8789.T':             prediction
 2023-04-22   64.283302
 2023-04-23   64.718216
 2023-04-24   64.687721
 2023-04-25   64.843407
 2023-04-26   64.916779,
 '1893.T':             prediction
 2023-04-22  665.432678
 2023-04-23  667.104797
 2023-04-24  665.367859
 2023-04-25  664.172058
 2023-04-26  666.226624,
 'MSFT':             prediction
 2023-04-22  241.574600
 2023-04-23  241.755371
 2023-04-24  240.824188
 2023-04-25  239.534119
 2023-04-26  239.983749}

In [135]:
display_prediction(target_symbols, predictions, actuals, future_predictions)